In [ ]:
!pip install catboost

In [ ]:
!pip install feature_engine

In [ ]:
import pandas as pd
import numpy as np
import warnings
import joblib
import plotly.express as px
from google.colab import drive
from datetime import date, timedelta
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.model_selection import train_test_split
# from plot_metric.functions import BinaryClassification
from datetime import datetime
import random
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import DBSCAN, MeanShift
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.metrics import silhouette_score, davies_bouldin_score

warnings.filterwarnings('ignore')
pd.options.display.max_columns = 500

from google.colab import auth
import gspread
from google.auth import default

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
drive.mount('/content/drive',  force_remount=True)

Mounted at /content/drive


In [ ]:
import json
data = {
  "account": "",
  "client_id": "764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com",
  "client_secret": "d-FL95Q19q7MQmFpd7hHD0Ty",
  "quota_project_id": "dev-sd-lake",
  "refresh_token": "1//05nz9IRnR1e2JCgYIARAAGAUSNwF-L9Ir_zi7EkfNFJb02e5XxL3e4Qicf7BN708CjMo66DuPPMhkd09jS6lIra-zubUPFGORcyU",
  "type": "authorized_user",
  "universe_domain": "googleapis.com"
}
json_data = json.dumps(data)
!echo '{json_data}' > '/content/.config/application_default_credentials.json'

In [ ]:
from google.cloud import bigquery

client = bigquery.Client(project='dev-sd-lake')

category_total_shelf_life_query = """
SELECT * FROM pollen.category_total_shelf_life;
"""

category_total_shelf_life = client.query_and_wait(category_total_shelf_life_query).to_dataframe()
category_total_shelf_life = category_total_shelf_life.rename(columns=category_total_shelf_life.iloc[0]).iloc[1:].reset_index(drop = True)

In [ ]:
from google.cloud import bigquery

client = bigquery.Client(project='dev-sd-lake')

product_listings_query = """
SELECT * FROM pollen.product_listings;
"""

product_listings = client.query_and_wait(product_listings_query).to_dataframe()


client = bigquery.Client(project='dev-sd-lake')

products_query = """
SELECT * FROM pollen.products;
"""

products = client.query_and_wait(products_query).to_dataframe()


client = bigquery.Client(project='dev-sd-lake')

product_categories_query = """
SELECT * FROM pollen.product_categories;
"""

product_categories = client.query_and_wait(product_categories_query).to_dataframe()


client = bigquery.Client(project='dev-sd-lake')

product_subcategories_query = """
SELECT * FROM pollen.product_subcategories;
"""

product_subcategories = client.query_and_wait(product_subcategories_query).to_dataframe()

client = bigquery.Client(project='dev-sd-lake')

sellers_query = """
SELECT * FROM pollen.sellers;
"""

sellers = client.query_and_wait(sellers_query).to_dataframe()

In [ ]:
offers_query = """
SELECT * FROM pollen.offers;
"""

offers = client.query_and_wait(offers_query).to_dataframe()

In [ ]:
orders_level_1_query = """
SELECT * FROM pollen.orders_level_1;
"""

orders_level_1 = client.query_and_wait(orders_level_1_query).to_dataframe()

In [ ]:
orders_level_2_query = """
SELECT * FROM pollen.orders_level_2;
"""

orders_level_2 = client.query_and_wait(orders_level_2_query).to_dataframe()

In [ ]:
sellers = sellers[['seller_name', 'persona_seller_type']].drop_duplicates()

In [ ]:
print(product_listings.shape, products.shape, product_categories.shape, product_subcategories.shape, sellers.shape)

(182283, 41) (182291, 9) (30, 3) (223, 4) (109, 2)


In [ ]:
def get_order_date(df_order_sheet1, df_order_sheet2):
  order_date_to_tracking_no_mapping = df_order_sheet1[['tracking_no', 'order_date']].drop_duplicates()
  df_order_sheet2 = pd.merge(df_order_sheet2, order_date_to_tracking_no_mapping, 'left',  on = ['tracking_no'])
  df_order_sheet2.drop('date_of_order', axis = 1, inplace = True)
  df_order = df_order_sheet2.copy()
  return df_order

orders = get_order_date(orders_level_1, orders_level_2)

In [ ]:
orders = orders.drop_duplicates(subset = ['sku_number', 'expiry_date', 'tracking_no'])
offers = offers.drop_duplicates(subset = ['sku_number', 'expiry_date', 'tracking_no'])
offers = offers.rename(columns = {'total_units' : 'total_units_offered'})
orders = orders.rename(columns = {'total_units' : 'total_units_ordered', 'total_offer_price_usd' : 'total_order_price_usd'})

offers = pd.merge(offers, orders[['sku_number', 'expiry_date', 'tracking_no', 'total_units_ordered', 'recovery_rate_percentage', 'total_order_price_usd']], 'left', on = ['sku_number', 'expiry_date', 'tracking_no'])

In [ ]:
product_subcategories = product_subcategories.rename(columns = {'id' : 'subcategory_id', 'name' : 'product_subcategory'})
product_categories = product_categories.rename(columns = {'id' : 'category_id', 'name' : 'product_category'})

products = pd.merge(products, product_categories, 'left', on = 'category_id')
products = pd.merge(products, product_subcategories[['subcategory_id', 'product_subcategory']], 'left', on = 'subcategory_id')

In [ ]:
products = products.rename(columns = {'id' : 'product_id'})
products = products.drop(['parent_product_id', 'created_at', 'updated_at', 'category_id', 'subcategory_id'], axis = 1)

product_listings = product_listings.drop(['created_at', 'updated_at', 'pollen_updated_price_per_unit_local',
                       'barcode', 'barcode_key', 'manufactured_date', 'batch_number', 'image_links', 'usd_conversion', 'scoring'], axis = 1)
product_listings = pd.merge(product_listings, products, 'left', on = 'product_id')

In [ ]:
product_listings.head(2)

,id,product_id,seller_id,record_id,seller_name,expiry_date,package_type,qty_of_carton,units_per_package_type,total_number_of_items,currency,retail_price_per_unit_local,asking_price_per_unit_local,pack_size_number,pack_size_unit,cbm_carton,length,breadth,height,unit_dimensions,weight_per_carton_kg,warehouse_address,warehouse_country,country_of_origin,package_labeled_language,product_restriction,priority,retail_price_per_unit_usd,asking_price_per_unit_usd,inventory_class,updated_on,sku_number,sku_product_name,brand,product_category,description,product_subcategory
0,2d61c777-e0d5-4744-b9eb-dd24f74ee061,44642cbd-8752-4f55-a368-669f22e9e8f8,SE1712631761502,SE1712631761502G4311200,L'Oreal Indonesia,2025-01-01,Carton,64.20,60.0,3852.0,IDR,24900.0,5991.5,31.0,g,0.01,254.0,156.0,219.0,MM,2.24,Bekasi,Indonesia,-,-,NaN,P2,1.53,0.37,CORE,2024-05-28,G4311200,GRN 2 STEP PW MASK B5 GOJI EB,GARNIER,SKIN CARE,SKIN CARE,MASKS & EXFOLIATORS
1,41508680-7c85-47aa-a54c-2bcbbddb56e9,6ca31813-8cac-4459-a53e-d68b62bf337d,SE1712631761502,SE1712631761502G4218000,L'Oreal Indonesia,2025-01-28,Carton,58.43,60.0,3506.0,IDR,23400.0,6552.7,19.7,g,0.01,185.0,150.0,197.0,MM,1.42,Bekasi,Indonesia,-,-,NaN,P2,1.44,0.40,CORE,2024-05-28,G4218000,GRN 5 MIN AMP MASK WATERMELON EB,GARNIER,SKIN CARE,SKIN CARE,MASKS & EXFOLIATORS


In [ ]:
product_listings.inventory_class.value_counts()

,count
inventory_class,
NON_CORE,115580
CORE,66703


In [ ]:
df_inv = product_listings[product_listings.inventory_class == 'CORE']
non_core_inv = product_listings[product_listings.inventory_class == 'NON_CORE']
df_inv = df_inv.reset_index(drop = True)
non_core_inv = non_core_inv.reset_index(drop = True)

In [ ]:
cols_list_underscore_cleaning = ['sku_product_name', 'brand', 'product_category', 'product_subcategory', 'warehouse_country',
            'country_of_origin', 'pack_label_language', 'product_restricted_countries', 'shelf_life_bucket']

inv_cols_list_to_change_dtypes = ['qty_of_cartons', 'units_per_cartons', 'retail_price_per_case_(local)', 'retail_price_per_case_(usd)', 'asking_price_per_case_(local)',
                              'asking_price_per_case_(usd)', 'total_retail_price_(local)', 'total_retail_price_(usd)', 'total_asking_price_(local)',
                              'total_asking_price_(usd)', 'pack_size_(number)', 'total_cbm', 'cbm_per_case', 'package_dimensions_per_package_type_(length)',
                              'package_dimensions_per_case_(width)', 'package_dimensions_per_case_(height)', 'net_weight_\nper_unit(kg)', 'net_weight_per_case_(kg)',
                              'gross_weight_per_case_(kg)', 'total_net_weight_(kg)', 'total_gross_weight_(kg)', 'cases_per_pallet', 'number_of_pallets', 'discount',
                              'shelf_remaining_days']

record_attributes = ['sku_number', 'sku_name', 'brand', 'product_category', 'product_subcategory', 'retail_price_per_unit_local', 'currency']

n_days_before_date = pd.to_datetime(date.today() - timedelta(days = 14))

In [ ]:
def dropping_null_brand_cat_subcats(df_inv):
  df_inv = df_inv[~df_inv.brand.isnull()]
  df_inv = df_inv[~df_inv.product_category.isnull()]
  df_inv = df_inv[~df_inv.product_subcategory.isnull()]
  return df_inv

def cols_to_lower_rem_space(df, cols_list):
  for col in ['sku_name', 'product_name', 'brand', 'brand_', 'product_description', 'product_category', 'product_sub_category', 'product_subcategory',\
              'warehouse_location', 'buyer', 'seller', 'package_type', 'deal_type', 'origin_', 'destination', 'region_of_export', \
              'domestic_export', 'order_type', 'country', 'currency', 'country_of_origin', 'pack_label_language', 'dangerous_goods\n(y/n)', 'shelf_life_bucket',
              'relavant_(r)/ir-relavant(ir)', 'product_restricted_countries', 'store_name', 'quarter', 'temp_reference', 'tracking_no', 'offer_type', 'fiscal_month',
              'proposed_buyer_product_preferences', 'product_shelf_life_months', 'priority', 'seller', 'lms_seller_id', 'persona', 'core_vs._\nnon_core', 'sku_product_name',
              'warehouse_country', 'warehouse_address', 'measurement_units', 'listing_currency', 'manufacturing_country', 'lbh_measurement_units', 'package_type', 'seller_name',
              'pack_size_unit', 'package_labeled_language', 'inventory_class', 'description', 'unit_dimensions']:
              if col in cols_list:
                df[col] = df[col].str.lower().str.replace(' ', '_')
  return df

def remove_starting_and_trailing_underscores(x):
  if x != '':
    if x[0] == '_':
      for i in range(len(x)):
        if x[i] != '_':
          start = i
          break
      x = x[start:]
    if x[-1] == '_':
      for i in reversed(range(len(x))):
        if x[i] != '_':
          end = i
          break
      x = x[:end+1]
  return x

def cleaning_underscores_inv_data(df_inv, col_list):
  for col in col_list:
    print(col)
    df_inv[col] = df_inv[col].apply(remove_starting_and_trailing_underscores)
  return df_inv

def drop_dups_and_clean(df_inv):
  print('Dropping above duplicates ...')
  df_inv = df_inv.drop_duplicates()
  return df_inv

In [ ]:
df_inv = dropping_null_brand_cat_subcats(df_inv)
df_inv = cols_to_lower_rem_space(df_inv, df_inv.columns)
df_inv = drop_dups_and_clean(df_inv)

non_core_inv = cols_to_lower_rem_space(non_core_inv, non_core_inv.columns)

Dropping above duplicates ...


# Non Core

In [ ]:
non_core_inv.head(2)

,id,product_id,seller_id,record_id,seller_name,expiry_date,package_type,qty_of_carton,units_per_package_type,total_number_of_items,currency,retail_price_per_unit_local,asking_price_per_unit_local,pack_size_number,pack_size_unit,cbm_carton,length,breadth,height,unit_dimensions,weight_per_carton_kg,warehouse_address,warehouse_country,country_of_origin,package_labeled_language,product_restriction,priority,retail_price_per_unit_usd,asking_price_per_unit_usd,inventory_class,updated_on,sku_number,sku_product_name,brand,product_category,description,product_subcategory
0,b61c2bb9-f203-4447-bd21-34c8b2d8a03a,c741e020-cae7-4258-b71f-3bc338b38412,SE1712631761542,SE171263176154204R3I1MKJPD97H7OBZ60UTU1Z,acommerce_singapore,2023-05-31,pieces,NaN,NaN,6.0,sgd,19.9,1.99,NaN,None,0.0,10.0,10.0,10.0,cm,0.50,sg-acom01,None,None,None,NaN,p1,15.116504,1.511650,non_core,2023-01-25,04R3I1MKJPD97H7OBZ60UTU1Z,l'oreal_paris_color_riche_satin_105_nude_in_paris,l'oreal_paris,makeup,makeup,lip_makeup
1,329218a0-49c7-4e36-bfe7-e10d17de8dfc,0cec2d8c-024a-4929-87fa-358f264016b7,SE1712631761542,SE171263176154211YAP8LR1W3OQSFJGWCLTD1DQ,acommerce_singapore,2023-05-31,pieces,NaN,NaN,160.0,sgd,2.0,0.20,NaN,None,0.0,8.0,5.0,0.2,cm,0.15,sg-acom02,None,None,None,NaN,p1,1.519247,0.151925,non_core,2023-01-25,11YAP8LR1W3OQSFJGWCLTD1DQ,garnier_serum_mask_-_pomegranate_(replumping),garnier,skin_care,skin_care,masks_&_exfoliators


In [ ]:
non_core_inv = non_core_inv.rename(columns = {'total_number_of_items' : 'total_units'})
non_core_inv['updated_on'] = pd.to_datetime(non_core_inv['updated_on'])
non_core_inv['expiry_date'] = pd.to_datetime(non_core_inv['expiry_date'])

In [ ]:
print(non_core_inv.shape)
non_core_inv = non_core_inv.drop_duplicates(subset = ['sku_number', 'expiry_date', 'updated_on', 'seller_name'])
print(non_core_inv.shape)

(115580, 37)
(64433, 37)


In [ ]:
offers = offers.rename(columns = {'seller' : 'seller_name'})
offers['date_of_offer'] = pd.to_datetime(offers['date_of_offer'])
offers['seller_name'] = offers['seller_name'].str.lower()
offers['total_units_ordered'] = offers['total_units_ordered'].str.replace(',', '')
offers['total_units_ordered'] = offers['total_units_ordered'].astype(float)

In [ ]:
def merge_non_core_inv_and_offer(non_core_inv, df_offer):
  temp = pd.merge(non_core_inv, df_offer[['sku_number', 'seller_name', 'date_of_offer', 'actual', 'tracking_no', 'buyer', 'total_units_ordered',
                                          'recovery_rate_percentage', 'offer_price_per_unit_usd', 'total_order_price_usd', 'destination']], 'left', on = ['sku_number', 'seller_name'])
  # temp['updated_on'] = pd.to_datetime(temp['updated_on'])
  temp['offer_day_diff_updated_inv'] = (temp['date_of_offer'] - temp['updated_on']).dt.days
  # avoiding future entry from same sku and seller to get filtered out hence nulling it as will remain in the dataframe
  temp['offer_day_diff_updated_inv'] = np.where(temp.offer_day_diff_updated_inv < 0, np.nan, temp['offer_day_diff_updated_inv'])
  temp = temp[(temp.offer_day_diff_updated_inv >= 0) | (temp.offer_day_diff_updated_inv.isnull())]
  df_non_core_inv_min_offer_day_diff_updated_inv = temp.groupby(['sku_number', 'seller_name', 'date_of_offer']).agg({'offer_day_diff_updated_inv' : 'min'}).reset_index()\
                                        .rename(columns = {'offer_day_diff_updated_inv' : 'min_offer_day_diff_updated_inv'})
  temp = pd.merge(temp, df_non_core_inv_min_offer_day_diff_updated_inv, 'left', on = ['sku_number', 'seller_name', 'date_of_offer'])
  temp = temp[(temp.offer_day_diff_updated_inv == temp.min_offer_day_diff_updated_inv) | (temp.offer_day_diff_updated_inv.isnull())]
  temp.drop('min_offer_day_diff_updated_inv', axis = 1, inplace = True)
  return temp

In [ ]:
non_core_inv = merge_non_core_inv_and_offer(non_core_inv, offers)
# non_core_inv['sell_thru_rate'] = non_core_inv['total_units_ordered'] / non_core_inv['total_units']
# non_core_inv['sell_thru_rate'] = np.where(non_core_inv['sell_thru_rate'] > 1, 1, non_core_inv['sell_thru_rate'])

In [ ]:
non_core_inv['shelf_life_remaining_days'] = (non_core_inv['expiry_date'] - non_core_inv['updated_on']).dt.days

In [ ]:
df_cluster = non_core_inv[['seller_name', 'warehouse_country', 'priority', 'retail_price_per_unit_usd', 'asking_price_per_unit_usd',
        'brand', 'product_category', 'product_subcategory']]

categorical_cols = ['priority', 'warehouse_country', 'brand', 'product_category', 'product_subcategory', 'seller_name']
df_encoded = df_cluster.copy()
le_dict = {}

for col in categorical_cols:
    le = LabelEncoder()
    df_encoded[col] = df_encoded[col].astype(str)  # convert NaNs to string "nan"
    df_encoded[col] = le.fit_transform(df_encoded[col])
    le_dict[col] = le

# Step 2: Apply IterativeImputer for missing values
imputer = IterativeImputer(random_state=42)
X_imputed = imputer.fit_transform(df_encoded)

# Step 3: Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)



# Step 4: Run DBSCAN
model = DBSCAN(eps=0.5, min_samples=200)
# model = MeanShift(bin_seeding = True)
df_cluster['cluster'] = model.fit_predict(X_scaled)

labels = df_cluster['cluster'].values
n_clusters = len(set(labels)) - (1 if -1 in labels else 0)

if n_clusters >= 2:
    # Remove noise points for silhouette score
    mask = labels != -1
    score = silhouette_score(X_scaled[mask], labels[mask])
    print(f"Silhouette Score (excluding noise): {score:.4f}")
else:
    print("Silhouette Score cannot be computed (requires at least 2 clusters excluding noise).")


non_core_inv['cluster'] = df_cluster['cluster']
non_core_inv['cluster'].unique()

Silhouette Score (excluding noise): 0.8634


array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
       16, 17, 18, 19, 20, 21, 22, 23])

In [ ]:
non_core_inv['total_order_price_usd'] = non_core_inv['total_order_price_usd'].str.replace(',', '').astype(float)
non_core_inv['order_price_per_unit_usd'] = non_core_inv['total_order_price_usd'] / non_core_inv['total_units_ordered']

## Recommendation - measure depletion at some other level, exclude skus as many skus don't repeat and many rows are getting dropped

# Core

In [ ]:
offers = offers.rename(columns = {'seller_name' : 'seller_short'})

seller_name_to_short_form = {'unilever_indonesia' : 'ulid', 'unilever_malaysia' : 'ulmy', 'unilever_philippines' : 'ulph', 'unilever_thailand' : 'ulth', 'unilever_singapore' : 'ulsg',
                             "l'oreal_malaysia" : "lomy", "l'oreal_philippines" : 'loph', "l'oreal_thailand" : "loth", "l'oreal_india" : "loin", "l'oreal_indonesia" : "loid"}

df_inv['updated_on'] = pd.to_datetime(df_inv['updated_on'])

def apply_seller_short(x):
  if x in seller_name_to_short_form.keys():
    return seller_name_to_short_form[x]
  else:
    return x

df_inv['seller_short'] = df_inv['seller_name'].apply(apply_seller_short)

offers = offers[offers['expiry_date'] != '-']
offers['expiry_date'] = pd.to_datetime(offers['expiry_date'], format = "%d-%b-%Y")
df_inv['expiry_date'] = pd.to_datetime(df_inv['expiry_date'])

In [ ]:
def merge_inv_and_offer(df_inv, df_offer):
  df_inv = pd.merge(df_inv, df_offer[['sku_number', 'expiry_date', 'seller_short', 'date_of_offer', 'actual', 'tracking_no', 'buyer', 'total_units_ordered',
                                      'recovery_rate_percentage', 'offer_price_per_unit_usd', 'total_order_price_usd', 'destination']], 'left', on = ['sku_number', 'expiry_date', 'seller_short'])
  df_inv['offer_day_diff_updated_inv'] = (df_inv['date_of_offer'] - df_inv['updated_on']).dt.days
  df_inv = df_inv[(df_inv.offer_day_diff_updated_inv >= 0) | (df_inv.offer_day_diff_updated_inv.isnull())]
  df_min_offer_day_diff_updated_inv = df_inv.groupby(['sku_number', 'expiry_date', 'seller_short', 'date_of_offer']).agg({'offer_day_diff_updated_inv' : 'min'}).reset_index()\
                                      .rename(columns = {'offer_day_diff_updated_inv' : 'min_offer_day_diff_updated_inv'})
  df_inv = pd.merge(df_inv, df_min_offer_day_diff_updated_inv, 'left', on = ['sku_number', 'expiry_date', 'seller_short', 'date_of_offer'])
  df_inv = df_inv[(df_inv.offer_day_diff_updated_inv == df_inv.min_offer_day_diff_updated_inv) | (df_inv.offer_day_diff_updated_inv.isnull())]
  df_inv.drop('min_offer_day_diff_updated_inv', axis = 1, inplace = True)
  return df_inv

In [ ]:
df_inv = merge_inv_and_offer(df_inv, offers)
df_inv['total_units_ordered'] = df_inv['total_units_ordered'].fillna(0)
df_inv['shelf_life_remaining_days'] = (df_inv['expiry_date'] - df_inv['updated_on']).dt.days

In [ ]:
df_cluster = df_inv[['total_number_of_items', 'seller_name', 'warehouse_country', 'priority', 'retail_price_per_unit_usd', 'asking_price_per_unit_usd',
        'brand', 'product_category', 'product_subcategory']]

categorical_cols = ['priority', 'warehouse_country', 'brand', 'product_category', 'product_subcategory', 'seller_name']
df_encoded = df_cluster.copy()
le_dict = {}

for col in categorical_cols:
    le = LabelEncoder()
    df_encoded[col] = df_encoded[col].astype(str)  # convert NaNs to string "nan"
    df_encoded[col] = le.fit_transform(df_encoded[col])
    le_dict[col] = le

# Step 2: Apply IterativeImputer for missing values
imputer = IterativeImputer(random_state=42)
X_imputed = imputer.fit_transform(df_encoded)

# Step 3: Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)



# Step 4: Run DBSCAN
model = DBSCAN(eps=0.5, min_samples=200)
# model = MeanShift(bin_seeding = True)
df_cluster['cluster'] = model.fit_predict(X_scaled)

labels = df_cluster['cluster'].values
n_clusters = len(set(labels)) - (1 if -1 in labels else 0)

if n_clusters >= 2:
    # Remove noise points for silhouette score
    mask = labels != -1
    score = silhouette_score(X_scaled[mask], labels[mask])
    print(f"Silhouette Score (excluding noise): {score:.4f}")
else:
    print("Silhouette Score cannot be computed (requires at least 2 clusters excluding noise).")

Silhouette Score (excluding noise): 0.8465


In [ ]:
df_inv['cluster'] = df_cluster['cluster']
df_inv['cluster'].unique()

array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
       16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32,
       33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 47, 44, 45, 46, 48, 49,
       50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66,
       67, 68, 69, 70])

In [ ]:
temp = df_inv.groupby('cluster')['destination'].value_counts().rename('counts').reset_index()
temp = temp.groupby('cluster').destination.first().reset_index()
cluster_to_popular_destination_country = dict(zip(temp.cluster, temp.destination))
DEFAULT = df_inv['destination'].mode().values[0]

In [ ]:
def set_destination(x):
  if pd.isna(x['destination']) and x['cluster'] in cluster_to_popular_destination_country:
    return cluster_to_popular_destination_country[x['cluster']]
  elif pd.isna(x['destination']) and x['cluster'] not in cluster_to_popular_destination_country:
    return DEFAULT
  else:
    return x['destination']

df_inv['destination'] = df_inv.apply(set_destination, axis = 1)
df_inv = df_inv.drop('cluster', axis = 1)

In [ ]:
# dropping duplicates considering all columns except the ids
k = list(set(df_inv.columns) - set([i for i in df_inv.columns if i.endswith('id')]))
df_inv = df_inv.drop_duplicates(subset = k)

In [ ]:
df_inv = df_inv.rename(columns = {'total_number_of_items' : 'total_units'})
df_inv['updated_on'] = pd.to_datetime(df_inv['updated_on'])
df_inv['expiry_date'] = pd.to_datetime(df_inv['expiry_date'])
df_inv = df_inv[df_inv.sku_number != '']
df_inv['shelf_life_remaining_days'] = (df_inv['expiry_date'] - df_inv['updated_on']).dt.days

In [ ]:
df_inv['total_order_price_usd'] = df_inv['total_order_price_usd'].str.replace(',', '').astype(float)
df_inv['order_price_per_unit_usd'] = df_inv['total_order_price_usd'] / df_inv['total_units_ordered']

In [ ]:
non_core_inv = non_core_inv.drop('cluster', axis = 1)
df_inv = df_inv.drop('seller_short', axis = 1)

In [ ]:
total_inv = pd.concat([df_inv, non_core_inv], ignore_index=True)
total_inv = total_inv[~total_inv.asking_price_per_unit_usd.isnull()].reset_index(drop = True)
updated_on_list = total_inv['updated_on']

In [ ]:
total_inv = total_inv.drop(['id', 'product_id', 'seller_id', 'record_id', 'expiry_date', 'pack_size_unit', 'unit_dimensions', 'package_labeled_language', 'product_restriction',
                'updated_on', 'sku_product_name', 'date_of_offer', 'actual', 'tracking_no', 'buyer', 'total_units_ordered', 'recovery_rate_percentage', 'offer_price_per_unit_usd',
                'total_order_price_usd', 'offer_day_diff_updated_inv', 'country_of_origin'], axis = 1)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import LabelEncoder

def impute_missing_values_iterative_label(df: pd.DataFrame) -> pd.DataFrame:
    df_copy = df.copy()

    # Store label encoders to reverse transform later if needed
    label_encoders = {}

    # Identify categorical and numerical columns
    categorical_cols = df_copy.select_dtypes(include=['object']).columns
    numerical_cols = df_copy.select_dtypes(include=[np.number]).columns

    # Encode categorical columns using LabelEncoder
    for col in categorical_cols:
        le = LabelEncoder()
        # Fill NaNs with a placeholder before encoding
        df_copy[col] = df_copy[col].fillna("missing")
        df_copy[col] = le.fit_transform(df_copy[col].astype(str))
        label_encoders[col] = le

    # Apply IterativeImputer
    imputer = IterativeImputer(random_state=42)
    imputed_array = imputer.fit_transform(df_copy)

    # Reconstruct DataFrame
    imputed_df = pd.DataFrame(imputed_array, columns=df_copy.columns)

    # Decode label-encoded categorical columns (optional)
    for col in categorical_cols:
        le = label_encoders[col]
        # Round to nearest integer since imputation gives floats
        imputed_df[col] = np.round(imputed_df[col]).astype(int)
        imputed_df[col] = le.inverse_transform(imputed_df[col])

    return imputed_df

In [ ]:
imputed_total_inv = impute_missing_values_iterative_label(total_inv)

In [ ]:
# treating negative order_price

positive_price_imputed_total_inv = imputed_total_inv[imputed_total_inv.order_price_per_unit_usd > 0]
positive_price_imputed_total_inv = positive_price_imputed_total_inv.groupby(['brand', 'product_category', 'product_subcategory']).order_price_per_unit_usd.mean().reset_index().rename(
    columns = {'order_price_per_unit_usd' : 'mean_order_price_per_unit_usd'}
)

imputed_total_inv = pd.merge(imputed_total_inv, positive_price_imputed_total_inv, how = 'left', on = ['brand',	'product_category',	'product_subcategory'])
imputed_total_inv['order_price_per_unit_usd'] = np.where(imputed_total_inv['order_price_per_unit_usd'] < 0, imputed_total_inv['mean_order_price_per_unit_usd'],
                                                         imputed_total_inv['order_price_per_unit_usd'])
imputed_total_inv = imputed_total_inv.drop('mean_order_price_per_unit_usd', axis = 1)

In [ ]:
imputed_total_inv.sample()

,seller_name,package_type,qty_of_carton,units_per_package_type,total_units,currency,retail_price_per_unit_local,asking_price_per_unit_local,pack_size_number,cbm_carton,length,breadth,height,weight_per_carton_kg,warehouse_address,warehouse_country,priority,retail_price_per_unit_usd,asking_price_per_unit_usd,inventory_class,sku_number,brand,product_category,description,product_subcategory,destination,shelf_life_remaining_days,order_price_per_unit_usd
34528,l'oreal_thailand,carton,0.17,24.0,4.0,thb,449.0,59.7,104.0,0.0,195.0,148.0,124.0,2.73,l'oréal_thailand_distribution_center,thailand,p2,12.58,1.67,core,G3521101,maybelline,makeup,makeup,face_makeup,Thailand,301.0,1.713968


In [ ]:
df_cluster = imputed_total_inv[['seller_name', 'warehouse_country', 'priority', 'retail_price_per_unit_usd', 'asking_price_per_unit_usd',
        'brand', 'product_category', 'product_subcategory', 'shelf_life_remaining_days', 'currency']]

categorical_cols = ['priority', 'warehouse_country', 'brand', 'product_category', 'product_subcategory', 'seller_name', 'currency']
df_encoded = df_cluster.copy()
le_dict = {}

for col in categorical_cols:
    le = LabelEncoder()
    df_encoded[col] = df_encoded[col].astype(str)  # convert NaNs to string "nan"
    df_encoded[col] = le.fit_transform(df_encoded[col])
    le_dict[col] = le

# Step 2: Apply IterativeImputer for missing values
imputer = IterativeImputer(random_state=42)
X_imputed = imputer.fit_transform(df_encoded)

# Step 3: Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)



# Step 4: Run DBSCAN
model = DBSCAN(eps=0.5, min_samples=200)
# model = MeanShift(bin_seeding = True)
df_cluster['cluster'] = model.fit_predict(X_scaled)

labels = df_cluster['cluster'].values
n_clusters = len(set(labels)) - (1 if -1 in labels else 0)

if n_clusters >= 2:
    # Remove noise points for silhouette score
    mask = labels != -1
    score = silhouette_score(X_scaled[mask], labels[mask])
    print(f"Silhouette Score (excluding noise): {score:.4f}")
else:
    print("Silhouette Score cannot be computed (requires at least 2 clusters excluding noise).")


imputed_total_inv['cluster'] = df_cluster['cluster']
imputed_total_inv['cluster'].unique()

Silhouette Score (excluding noise): 0.8167


array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
       16, 23, 17, 18, 19, 20, 21, 22, 26, 25, 32, 24, 27, 28, 29, 44, 30,
       35, 31, 33, 34, 36, 37, 38, 39, 40, 41, 42, 43, 45, 46, 47, 48, 49,
       50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66,
       67, 68, 69, 70, 71, 72, 73, 74, 75])

In [ ]:
cluster_to_popular_source_destination = imputed_total_inv.groupby('cluster').agg({'destination' : pd.Series.mode, 'warehouse_country' : pd.Series.mode,
                        'priority' : pd.Series.mode}).reset_index().rename(columns = {'destination' : 'popular_destination', 'warehouse_country' : 'popular_warehouse_country',
                                                                                       'priority' : 'popular_priority'})
imputed_total_inv = pd.merge(imputed_total_inv, cluster_to_popular_source_destination, 'left', on = 'cluster')
imputed_total_inv['warehouse_country'] = np.where(imputed_total_inv['warehouse_country'] == 'missing', imputed_total_inv['popular_warehouse_country'], imputed_total_inv['warehouse_country'])
imputed_total_inv['destination'] = np.where(imputed_total_inv['destination'] == 'missing', imputed_total_inv['popular_destination'], imputed_total_inv['destination'])
imputed_total_inv['priority'] = np.where(imputed_total_inv['priority'] == 'missing', imputed_total_inv['popular_priority'], imputed_total_inv['priority'])

In [ ]:
UPPER_THRESH = imputed_total_inv["order_price_per_unit_usd"].quantile(0.85)
imputed_total_inv['order_price_per_unit_usd'] = np.where(imputed_total_inv['order_price_per_unit_usd'] > UPPER_THRESH, UPPER_THRESH,
                                                         imputed_total_inv['order_price_per_unit_usd'])
imputed_total_inv = imputed_total_inv.drop(['popular_destination', 'popular_warehouse_country', 'popular_priority'], axis = 1)

In [ ]:
imputed_total_inv['updated_on'] = updated_on_list

In [ ]:
print(imputed_total_inv['updated_on'].isnull().sum())
imputed_total_inv = imputed_total_inv[~imputed_total_inv['updated_on'].isnull()]
imputed_total_inv['destination'] = imputed_total_inv['destination'].str.lower().str.replace(' ', '_')

8


In [ ]:
imputed_total_inv['month'] = imputed_total_inv['updated_on'].dt.month
imputed_total_inv['year'] = imputed_total_inv['updated_on'].dt.year
imputed_total_inv['week'] = imputed_total_inv['updated_on'].dt.isocalendar().week
imputed_total_inv['year_month'] = imputed_total_inv['year'].astype(str) + '_' + imputed_total_inv['month'].astype(str)
imputed_total_inv['year_week'] = imputed_total_inv['year'].astype(str) + '_' + imputed_total_inv['week'].astype(str)

In [ ]:
imputed_total_inv['order_price_per_unit_usd'] = np.where(imputed_total_inv['order_price_per_unit_usd'] > imputed_total_inv['asking_price_per_unit_usd'],
         np.random.uniform(low = 0.7, high = 1) * imputed_total_inv['asking_price_per_unit_usd'],
         imputed_total_inv['order_price_per_unit_usd'])

In [ ]:
psi_df = imputed_total_inv[['sku_number', 'brand', 'product_category', 'product_subcategory', 'seller_name', 'updated_on',
                   'warehouse_country', 'destination', 'priority', 'order_price_per_unit_usd', 'asking_price_per_unit_usd', 'year_week', 'year_month']]

# Plots

## Cat month

In [ ]:
df_psi_cat_month = psi_df.groupby(['product_category', 'priority', 'warehouse_country', 'destination', 'year_month']).agg({'order_price_per_unit_usd' : 'mean',
                                                                                                                           'asking_price_per_unit_usd' : 'mean'}).reset_index()

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

def cat_month_add_dummy_rows(df):
    df = df.copy()

    # Convert year_month to datetime format
    df['year_month'] = pd.to_datetime(df['year_month'].str.replace('_', '-'), format='%Y-%m')
    today = pd.to_datetime(datetime.today().replace(day=1))
    prev_month = today - pd.DateOffset(months=1)

    key_cols = ['product_category', 'priority', 'warehouse_country', 'destination']
    all_combinations = df[key_cols].drop_duplicates()

    final_rows = []

    for _, combo in all_combinations.iterrows():
        combo_filter = (df[key_cols] == combo.values).all(axis=1)
        combo_df = df[combo_filter].sort_values('year_month')

        if combo_df.empty:
            continue

        start = combo_df['year_month'].min()
        months_needed = pd.date_range(start=start, end=prev_month, freq='MS')

        for month in months_needed:
            if month in combo_df['year_month'].values:
                row = combo_df[combo_df['year_month'] == month].iloc[0].to_dict()
                final_rows.append(row)
            else:
                prev_row = combo_df[combo_df['year_month'] < month].sort_values('year_month').tail(1)
                next_row = combo_df[combo_df['year_month'] > month].sort_values('year_month').head(1)

                if not prev_row.empty:
                    prev_order = prev_row['order_price_per_unit_usd'].values[0]
                    prev_asking = prev_row['asking_price_per_unit_usd'].values[0]
                else:
                    prev_order = prev_asking = 0.0

                if not next_row.empty:
                    next_order = next_row['order_price_per_unit_usd'].values[0]
                    next_asking = next_row['asking_price_per_unit_usd'].values[0]
                    try:
                        order_price = np.random.uniform(min(prev_order, next_order), max(prev_order, next_order))
                    except:
                        order_price = np.nan
                    try:
                        asking_price = np.random.uniform(min(prev_asking, next_asking), max(prev_asking, next_asking))
                    except:
                        asking_price = np.nan
                else:
                    try:
                        order_price = np.random.uniform(0.8 * prev_order, 1.2 * prev_order)
                    except:
                        order_price = np.nan
                    try:
                        asking_price = np.random.uniform(0.8 * prev_asking, 1.2 * prev_asking)
                    except:
                        asking_price = np.nan

                dummy_row = {
                    'product_category': combo['product_category'],
                    'priority': combo['priority'],
                    'warehouse_country': combo['warehouse_country'],
                    'destination': combo['destination'],
                    'year_month': month,
                    'order_price_per_unit_usd': order_price,
                    'asking_price_per_unit_usd': asking_price
                }
                final_rows.append(dummy_row)

    final_df = pd.DataFrame(final_rows)
    final_df['year_month'] = pd.to_datetime(final_df['year_month'])
    final_df['year_month'] = final_df['year_month'].dt.strftime('%Y_%m')
    final_df = final_df.sort_values(by=key_cols + ['year_month']).reset_index(drop=True)

    return final_df

In [ ]:
filled_df_psi_cat_month = cat_month_add_dummy_rows(df_psi_cat_month)

In [ ]:
plot_temp = filled_df_psi_cat_month[(filled_df_psi_cat_month['product_category'] == 'skin_care') & (filled_df_psi_cat_month['priority'] == 'p2') &
                                (filled_df_psi_cat_month['warehouse_country'] == 'thailand') & (filled_df_psi_cat_month['destination'] == 'hong_kong')]
plot_temp = plot_temp[-8:] # last 8 months plot
fig = px.line(plot_temp, x="year_month", y="order_price_per_unit_usd", title='PSI Cat Month')
fig.show()

In [ ]:
price_psi_cat_month_country_maps = filled_df_psi_cat_month.groupby(['product_category', 'priority', 'warehouse_country', 'destination']).agg({'order_price_per_unit_usd' : 'nunique'})\
.reset_index().rename(columns = {'order_price_per_unit_usd' : 'unique_prices'})
print(len(price_psi_cat_month_country_maps))
price_psi_cat_month_country_maps = price_psi_cat_month_country_maps[price_psi_cat_month_country_maps.unique_prices >= 8] # 8 months

price_psi_cat_month_country_maps = price_psi_cat_month_country_maps.drop('unique_prices', axis = 1)
print(len(price_psi_cat_month_country_maps))

573
464


## Cat week

In [ ]:
df_psi_cat_week = psi_df.groupby(['product_category', 'priority', 'warehouse_country', 'destination', 'year_week']).agg({'order_price_per_unit_usd' : 'mean',
                                                                                                                         'asking_price_per_unit_usd' : 'mean'}).reset_index()

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

def cat_week_add_dummy_rows(df):
    df = df.copy()

    # Convert 'year_week' to datetime format (Monday of that week)
    df['week_start'] = df['year_week'].apply(lambda x: datetime.strptime(f"{x}_1", "%Y_%W_%w"))

    today = datetime.today().replace(day=1)
    last_day_prev_month = today - timedelta(days=1)
    last_week_prev_month = last_day_prev_month - timedelta(days=last_day_prev_month.weekday())

    key_cols = ['product_category', 'priority', 'warehouse_country', 'destination']
    all_combinations = df[key_cols].drop_duplicates()

    final_rows = []

    for _, combo in all_combinations.iterrows():
        combo_filter = (df[key_cols] == combo.values).all(axis=1)
        combo_df = df[combo_filter].sort_values('week_start')

        if combo_df.empty:
            continue

        start_date = combo_df['week_start'].min()
        weeks_needed = pd.date_range(start=start_date, end=last_week_prev_month, freq='W-MON')

        for week_start in weeks_needed:
            if week_start in combo_df['week_start'].values:
                row = combo_df[combo_df['week_start'] == week_start].iloc[0].to_dict()
                final_rows.append(row)
            else:
                prev_row = combo_df[combo_df['week_start'] < week_start].sort_values('week_start').tail(1)
                next_row = combo_df[combo_df['week_start'] > week_start].sort_values('week_start').head(1)

                if not prev_row.empty:
                    prev_price = prev_row['order_price_per_unit_usd'].values[0]
                    prev_asking = prev_row['asking_price_per_unit_usd'].values[0]
                else:
                    prev_price = prev_asking = 0.0

                if not next_row.empty:
                    next_price = next_row['order_price_per_unit_usd'].values[0]
                    next_asking = next_row['asking_price_per_unit_usd'].values[0]
                    try:
                        order_price = np.random.uniform(min(prev_price, next_price), max(prev_price, next_price))
                    except:
                        order_price = np.nan
                    try:
                        asking_price = np.random.uniform(min(prev_asking, next_asking), max(prev_asking, next_asking))
                    except:
                        asking_price = np.nan
                else:
                    try:
                        order_price = np.random.uniform(0.8 * prev_price, 1.2 * prev_price)
                    except:
                        order_price = np.nan
                    try:
                        asking_price = np.random.uniform(0.8 * prev_asking, 1.2 * prev_asking)
                    except:
                        asking_price = np.nan

                dummy_row = {
                    'product_category': combo['product_category'],
                    'priority': combo['priority'],
                    'warehouse_country': combo['warehouse_country'],
                    'destination': combo['destination'],
                    'week_start': week_start,
                    'order_price_per_unit_usd': order_price,
                    'asking_price_per_unit_usd': asking_price

                }
                final_rows.append(dummy_row)

    final_df = pd.DataFrame(final_rows)
    final_df['year_week'] = final_df['week_start'].dt.strftime('%Y_%W')
    final_df = final_df.drop(columns='week_start')
    final_df = final_df.sort_values(by=key_cols + ['year_week']).reset_index(drop=True)

    return final_df

In [ ]:
filled_df_psi_cat_week = cat_week_add_dummy_rows(df_psi_cat_week)

In [ ]:
plot_temp = filled_df_psi_cat_week[(filled_df_psi_cat_week['product_category'] == 'hair_care') & (filled_df_psi_cat_week['priority'] == 'p2') &
                                (filled_df_psi_cat_week['warehouse_country'] == 'thailand') & (filled_df_psi_cat_week['destination'] == 'thailand')]
plot_temp = plot_temp[-32:] # last 8 months plot
fig = px.line(plot_temp, x="year_week", y="order_price_per_unit_usd", title='PSI Cat Week')
fig.show()

In [ ]:
price_psi_cat_week_country_maps = filled_df_psi_cat_week.groupby(['product_category', 'priority', 'warehouse_country', 'destination']).agg({'order_price_per_unit_usd' : 'nunique'})\
.reset_index().rename(columns = {'order_price_per_unit_usd' : 'unique_prices'})
print(len(price_psi_cat_week_country_maps))
price_psi_cat_week_country_maps = price_psi_cat_week_country_maps[price_psi_cat_week_country_maps.unique_prices >= 32] # 8 months

price_psi_cat_week_country_maps = price_psi_cat_week_country_maps.drop('unique_prices', axis = 1)
print(len(price_psi_cat_week_country_maps))

573
464


## Subcat month

In [ ]:
df_psi_subcat_month = psi_df.groupby(['product_subcategory', 'priority', 'warehouse_country', 'destination', 'year_month']).agg({'order_price_per_unit_usd' : 'mean',
                                                                                                                                 'asking_price_per_unit_usd' : 'mean'}).reset_index()

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

def subcat_month_add_dummy_rows(df):
    df = df.copy()

    # Convert year_month to datetime format
    df['year_month'] = pd.to_datetime(df['year_month'].str.replace('_', '-'), format='%Y-%m')
    today = pd.to_datetime(datetime.today().replace(day=1))
    prev_month = today - pd.DateOffset(months=1)

    key_cols = ['product_subcategory', 'priority', 'warehouse_country', 'destination']
    all_combinations = df[key_cols].drop_duplicates()

    final_rows = []

    for _, combo in all_combinations.iterrows():
        combo_filter = (df[key_cols] == combo.values).all(axis=1)
        combo_df = df[combo_filter].sort_values('year_month')

        if combo_df.empty:
            continue

        start = combo_df['year_month'].min()
        months_needed = pd.date_range(start=start, end=prev_month, freq='MS')

        for month in months_needed:
            if month in combo_df['year_month'].values:
                row = combo_df[combo_df['year_month'] == month].iloc[0].to_dict()
                final_rows.append(row)
            else:
                prev_row = combo_df[combo_df['year_month'] < month].sort_values('year_month').tail(1)
                next_row = combo_df[combo_df['year_month'] > month].sort_values('year_month').head(1)

                if not prev_row.empty:
                    prev_order = prev_row['order_price_per_unit_usd'].values[0]
                    prev_asking = prev_row['asking_price_per_unit_usd'].values[0]
                else:
                    prev_order = prev_asking = 0.0

                if not next_row.empty:
                    next_order = next_row['order_price_per_unit_usd'].values[0]
                    next_asking = next_row['asking_price_per_unit_usd'].values[0]
                    try:
                        order_price = np.random.uniform(min(prev_order, next_order), max(prev_order, next_order))
                    except:
                        order_price = np.nan
                    try:
                        asking_price = np.random.uniform(min(prev_asking, next_asking), max(prev_asking, next_asking))
                    except:
                        asking_price = np.nan
                else:
                    try:
                        order_price = np.random.uniform(0.8 * prev_order, 1.2 * prev_order)
                    except:
                        order_price = np.nan
                    try:
                        asking_price = np.random.uniform(0.8 * prev_asking, 1.2 * prev_asking)
                    except:
                        asking_price = np.nan

                dummy_row = {
                    'product_subcategory': combo['product_subcategory'],
                    'priority': combo['priority'],
                    'warehouse_country': combo['warehouse_country'],
                    'destination': combo['destination'],
                    'year_month': month,
                    'order_price_per_unit_usd': order_price,
                    'asking_price_per_unit_usd': asking_price
                }
                final_rows.append(dummy_row)

    final_df = pd.DataFrame(final_rows)
    final_df['year_month'] = pd.to_datetime(final_df['year_month'])
    final_df['year_month'] = final_df['year_month'].dt.strftime('%Y_%m')
    final_df = final_df.sort_values(by=key_cols + ['year_month']).reset_index(drop=True)

    return final_df

In [ ]:
filled_df_psi_subcat_month = subcat_month_add_dummy_rows(df_psi_subcat_month)

In [ ]:
plot_temp = filled_df_psi_subcat_month[(filled_df_psi_subcat_month['product_subcategory'] == 'shampoo') & (filled_df_psi_subcat_month['priority'] == 'p1') &
                                (filled_df_psi_subcat_month['warehouse_country'] == 'indonesia') & (filled_df_psi_subcat_month['destination'] == 'indonesia')]
plot_temp = plot_temp[-8:] # last 8 months plot
fig = px.line(plot_temp, x="year_month", y="order_price_per_unit_usd", title='PSI Subcat Month')
fig.show()

In [ ]:
price_psi_subcat_month_country_maps = filled_df_psi_subcat_month.groupby(['product_subcategory', 'priority', 'warehouse_country', 'destination']).agg({'order_price_per_unit_usd' : 'nunique'})\
.reset_index().rename(columns = {'order_price_per_unit_usd' : 'unique_prices'})
print(len(price_psi_subcat_month_country_maps))
price_psi_subcat_month_country_maps = price_psi_subcat_month_country_maps[price_psi_subcat_month_country_maps.unique_prices >= 8] # 8 months

price_psi_subcat_month_country_maps = price_psi_subcat_month_country_maps.drop('unique_prices', axis = 1)
print(len(price_psi_subcat_month_country_maps))

1392
1097


# Subcat week

In [ ]:
df_psi_subcat_week = psi_df.groupby(['product_subcategory', 'priority', 'warehouse_country', 'destination', 'year_week']).agg({'order_price_per_unit_usd' : 'mean',
                                                                                                                               'asking_price_per_unit_usd' : 'mean'}).reset_index()

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

def subcat_week_add_dummy_rows(df):
    df = df.copy()

    # Convert 'year_week' to datetime format (Monday of that week)
    df['week_start'] = df['year_week'].apply(lambda x: datetime.strptime(f"{x}_1", "%Y_%W_%w"))

    today = datetime.today().replace(day=1)
    last_day_prev_month = today - timedelta(days=1)
    last_week_prev_month = last_day_prev_month - timedelta(days=last_day_prev_month.weekday())

    key_cols = ['product_subcategory', 'priority', 'warehouse_country', 'destination']
    all_combinations = df[key_cols].drop_duplicates()

    final_rows = []

    for _, combo in all_combinations.iterrows():
        combo_filter = (df[key_cols] == combo.values).all(axis=1)
        combo_df = df[combo_filter].sort_values('week_start')

        if combo_df.empty:
            continue

        start_date = combo_df['week_start'].min()
        weeks_needed = pd.date_range(start=start_date, end=last_week_prev_month, freq='W-MON')

        for week_start in weeks_needed:
            if week_start in combo_df['week_start'].values:
                row = combo_df[combo_df['week_start'] == week_start].iloc[0].to_dict()
                final_rows.append(row)
            else:
                prev_row = combo_df[combo_df['week_start'] < week_start].sort_values('week_start').tail(1)
                next_row = combo_df[combo_df['week_start'] > week_start].sort_values('week_start').head(1)

                if not prev_row.empty:
                    prev_price = prev_row['order_price_per_unit_usd'].values[0]
                    prev_asking = prev_row['asking_price_per_unit_usd'].values[0]
                else:
                    prev_price = prev_asking = 0.0

                if not next_row.empty:
                    next_price = next_row['order_price_per_unit_usd'].values[0]
                    next_asking = next_row['asking_price_per_unit_usd'].values[0]
                    try:
                        order_price = np.random.uniform(min(prev_price, next_price), max(prev_price, next_price))
                    except:
                        order_price = np.nan
                    try:
                        asking_price = np.random.uniform(min(prev_asking, next_asking), max(prev_asking, next_asking))
                    except:
                        asking_price = np.nan
                else:
                    try:
                        order_price = np.random.uniform(0.8 * prev_price, 1.05 * prev_price)
                    except:
                        order_price = np.nan
                    try:
                        asking_price = np.random.uniform(0.8 * prev_asking, 1.05 * prev_asking)
                    except:
                        asking_price = np.nan

                dummy_row = {
                    'product_subcategory': combo['product_subcategory'],
                    'priority': combo['priority'],
                    'warehouse_country': combo['warehouse_country'],
                    'destination': combo['destination'],
                    'week_start': week_start,
                    'order_price_per_unit_usd': order_price,
                    'asking_price_per_unit_usd': asking_price
                }
                final_rows.append(dummy_row)

    final_df = pd.DataFrame(final_rows)
    final_df['year_week'] = final_df['week_start'].dt.strftime('%Y_%W')
    final_df = final_df.drop(columns='week_start')
    final_df = final_df.sort_values(by=key_cols + ['year_week']).reset_index(drop=True)

    return final_df

In [ ]:
filled_df_psi_subcat_week = subcat_week_add_dummy_rows(df_psi_subcat_week)

In [ ]:
plot_temp = filled_df_psi_subcat_week[(filled_df_psi_subcat_week['product_subcategory'] == 'face_makeup') & (filled_df_psi_subcat_week['priority'] == 'p2') &
                                (filled_df_psi_subcat_week['warehouse_country'] == 'indonesia') & (filled_df_psi_subcat_week['destination'] == 'indonesia')]
plot_temp = plot_temp[-32:] # last 8 months plot
fig = px.line(plot_temp, x="year_week", y="order_price_per_unit_usd", title='PSI Subcat Week')
fig.show()

In [ ]:
price_psi_subcat_week_country_maps = filled_df_psi_subcat_week.groupby(['product_subcategory', 'priority', 'warehouse_country', 'destination']).agg({'order_price_per_unit_usd' : 'nunique'})\
.reset_index().rename(columns = {'order_price_per_unit_usd' : 'unique_prices'})
print(len(price_psi_subcat_week_country_maps))
price_psi_subcat_week_country_maps = price_psi_subcat_week_country_maps[price_psi_subcat_week_country_maps.unique_prices >= 32] # 8 months

price_psi_subcat_week_country_maps = price_psi_subcat_week_country_maps.drop('unique_prices', axis = 1)
print(len(price_psi_subcat_week_country_maps))

1392
1097


In [ ]:
# def dump_files_for_inference_pipeline():
#   print('Dumping files for inference later ...')
#   joblib.dump(filled_df_psi_cat_month, '/content/drive/MyDrive/PSI_Modeling_Data/PSI/price/filled_df_psi_cat_month.pkl')
#   joblib.dump(filled_df_psi_cat_week, '/content/drive/MyDrive/PSI_Modeling_Data/PSI/price/filled_df_psi_cat_week.pkl')
#   joblib.dump(filled_df_psi_subcat_month, '/content/drive/MyDrive/PSI_Modeling_Data/PSI/price/filled_df_psi_subcat_month.pkl')
#   joblib.dump(filled_df_psi_subcat_week, '/content/drive/MyDrive/PSI_Modeling_Data/PSI/price/filled_df_psi_subcat_week.pkl')
#   joblib.dump(price_psi_cat_month_country_maps, '/content/drive/MyDrive/PSI_Modeling_Data/PSI/price/price_psi_cat_month_country_maps.pkl')
#   joblib.dump(price_psi_cat_week_country_maps, '/content/drive/MyDrive/PSI_Modeling_Data/PSI/price/price_psi_cat_week_country_maps.pkl')
#   joblib.dump(price_psi_subcat_month_country_maps, '/content/drive/MyDrive/PSI_Modeling_Data/PSI/price/price_psi_subcat_month_country_maps.pkl')
#   joblib.dump(price_psi_subcat_week_country_maps, '/content/drive/MyDrive/PSI_Modeling_Data/PSI/price/price_psi_subcat_week_country_maps.pkl')

In [ ]:
# dump_files_for_inference_pipeline()